In [18]:
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
import pandas as pd
import time
import json
from bs4 import BeautifulSoup

In [ ]:
def scrape_all_events():
  """
  Scrape all events from Parimatch for a given sport.

  Args:
    sport: The sport to scrape data for.

  Returns:
    A DataFrame containing the scraped data.
  """

  # Get the URL for the events page.
  url = f"https://parimatch-in.com/en/cricket/"
  time.sleep(2)
  # Scrape the page.
  response = requests.get(url)
  time.sleep(2)
  soup = BeautifulSoup(response.content, "html.parser")

  # Get the events.
  events = soup.find_all("div", class_="styles_wrapper__BfdYz styles_wrapper-card-with-favorites__kCilz styles_wrapper-card__2uN8L EC_Hb")

  # Create a DataFrame to store the data.
  df = pd.DataFrame()

  # Add the events to the DataFrame.
  for event in events:
    event_name = event.find("h3").text
    event_url = event.find("a").get("href")
    df = df.append({"Event Name": event_name, "Event URL": event_url}, ignore_index=True)

  # Return the DataFrame.
  return events

In [19]:
import requests
import time
from bs4 import BeautifulSoup
url = f"https://parimatch-in.com/en/cricket/"
# Scrape the page.
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
time.sleep(2)
print(soup)
events = soup.find_all("div", class_="styles_wrapper__BfdYz styles_wrapper-card-with-favorites__kCilz styles_wrapper-card__2uN8L EC_Hb")
print(events)

<!DOCTYPE html>
<html lang="en-US"><head><meta charset="utf-8"/><title>...</title><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0" name="viewport"/><meta content="#000000" name="theme-color"/><link href="pwa/manifest.json" rel="manifest"/><link href="/favicon.ico" rel="shortcut icon"/><link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="/content/uploads/icons/IN/iphone5_splash.png" media="(device-width: 320px) and (device-height: 568px) and (-webkit-device-pixel-ratio: 2)" rel="apple-touch-startup-image"/><link href="/content/uploads/icons/IN/iphone6_splash.png" media="(device-width: 375px) and (device-height: 667px) and (-webkit-device-pixel-ratio: 2)" rel="apple-touch-startup-image"/><link href="/content/uploads/icons/IN/iphoneplus_splash.png" media="(device-width: 621px) and (device-height: 1104px) and (-webkit-device-pixel-ratio: 3)" rel="ap

In [57]:
import requests
import time
from bs4 import BeautifulSoup
url = f"https://parimatch-in.com/en/cricket/live"
headers = {
    'User-agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}
time.sleep(5)
response = requests.get(url,headers)
time.sleep(5)
soup = BeautifulSoup(response.content, "html.parser")


In [58]:
print(response)

<Response [200]>


In [59]:
soup.select("line-holder > div.EC_AI")

[]

In [60]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="en-US">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   ...\n  </title>\n  <meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0" name="viewport"/>\n  <meta content="#000000" name="theme-color"/>\n  <link href="pwa/manifest.json" rel="manifest"/>\n  <link href="/favicon.ico" rel="shortcut icon"/>\n  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>\n  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>\n  <link href="/content/uploads/icons/IN/iphone5_splash.png" media="(device-width: 320px) and (device-height: 568px) and (-webkit-device-pixel-ratio: 2)" rel="apple-touch-startup-image"/>\n  <link href="/content/uploads/icons/IN/iphone6_splash.png" media="(device-width: 375px) and (device-height: 667px) and (-webkit-device-pixel-ratio: 2)" rel="apple-touch-startup-image"/>\n  <link href="/content/uploads/icons/IN/iphoneplus_splash.png" media="(device-width: 621px) and (device-he

In [44]:
events=soup.findAll("div",id="root")

TypeError: Tag.childGenerator() takes 1 positional argument but 2 were given

In [ ]:
print(response.content)

In [ ]:
print(events)

In [ ]:
df1 = scrape_all_events()
print(df1)

In [33]:
import requests
from bs4 import BeautifulSoup

url = "https://parimatch.com/en/sports/cricket/"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

matches = soup.find_all("div", class_="match-item")

for match in matches:

    match_name = match.find("h3").text
    match_date = match.find("span", class_="match-date").text
    match_time = match.find("span", class_="match-time").text
    team1 = match.find("span", class_="team1").text
    team2 = match.find("span", class_="team2").text
    odds = match.find("span", class_="odds").text

    print(f"{match_name} | {match_date} | {match_time} | {team1} | {team2} | {odds}")


In [ ]:
print(df)

In [ ]:
browser = webdriver.Firefox()
browser.get('https://gg.bet/en/live')

In [ ]:
browser.get('https://gg.bet/en/live')
html=browser.page_source
time.sleep(3)
table = browser.find_element(By.CLASS_NAME, "overviewRow__container___2uPYc")

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
for a in soup:
    print(a)

In [ ]:
a=table.get_attribute('outerHTML')
BeautifulSoup.prettify(a)

In [ ]:
def scrape_surebets():
    browser = webdriver.Firefox()
    browser.get('https://en.surebet.com/surebets')
    html=browser.page_source
    time.sleep(10)
    table = browser.find_element(By.CSS_SELECTOR, "#surebets-table")
    table_html = table.get_attribute('outerHTML')
    df = pd.read_html(table_html)[0]
    df.drop(columns=["▼Profit","Unnamed: 7","Unnamed: 8"])
    browser.close()
    return df

In [ ]:
df=scrape_surebets()

In [ ]:
df

In [ ]:
df.to_csv('/media/adi/032E-9FF0/projects/web bets/surebets.csv', index=False)

In [ ]:
a=soup.find("class=__app-LogoTitle-name logoTitle__name___3_ywM")

In [ ]:
print(a)

In [ ]:
df.drop(columns=["▼Profit","Unnamed: 7","Unnamed: 8"]).to_records

In [ ]:
df

In [ ]:
"meta"

In [ ]:
df.to_records()

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://parimatch.com/en/sports/cricket/"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

matches = soup.find_all("div", class_="match-item")

for match in matches:

    match_name = match.find("h3").text
    match_date = match.find("span", class_="match-date").text
    match_time = match.find("span", class_="match-time").text
    team1 = match.find("span", class_="team1").text
    team2 = match.find("span", class_="team2").text
    odds = match.find("span", class_="odds").text

    print(f"{match_name} | {match_date} | {match_time} | {team1} | {team2} | {odds}")


In [ ]:
import json
import re
import time
from datetime import datetime

import requests

headers = {
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-US,en;q=0.9,pl;q=0.8",
    "referer": "https://www.oddsportal.com/",
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.99 Safari/537.36",
}

def get_response(url: str) -> str:
    return requests.get(url, headers=headers).text

time_now_s = int(time.time())
time_now_ms = int(round(time.time() * 1000))

bookies_js = f"https://www.oddsportal.com/res/x/bookies-201014103652-{time_now_s}.js"
odds_data_js = f"https://fb.oddsportal.com/feed/match/1-1-ld9FDhEI-1-2-yje1c.dat?_={time_now_ms}"

bookies = json.loads(re.findall(r"bookmakersData=(\{.*\});var", get_response(bookies_js))[0])
odds_data = json.loads(re.findall(r"\.dat',\s(\{.*\})", get_response(odds_data_js))[0])

bookie = bookies["417"]["WebName"]  # 417 is 1xBet's code
bookies_odds = odds_data["d"]["oddsdata"]["back"]["E-1-2-0-0-0"]["odds"]["417"]  # current odds for a bookie
odds_sorted = dict(sorted(bookies_odds.items())).values()  # sorted as on the website 1 - X - 2

print(f"{bookie} - {' '.join(str(i) for i in odds_sorted)}")

history_columns = {
    "1": "4ccecxv464x0xbcsm1",  # 1 column
    "2": "4ccecxv464x0xbcsm2",  # 2 column
    "X": "4ccecxv498x0x0",  # X column
}

# odds history for the X column for a given bookie
history_data = odds_data["d"]["history"]["back"][history_columns["X"]]["417"]
for item in history_data:
    value, _, timestamp = item
    print(f"{datetime.fromtimestamp(timestamp)} - {value}")

In [ ]:
import requests
from bs4 import BeautifulSoup

# Specify the URL of the page you want to scrape
url = 'https://parimatch-in.com/en/cricket/'

# Make a request to the URL
response = requests.get(url)

# Create a BeautifulSoup object from the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the elements that contain the data you want to scrape
elements = soup.find_all('div')

# Loop over the elements and extract the data
for element in elements:
    # Get the name of the event
    event_name = element.find('h3')

    # Get the time of the event
    event_time = element.find('span', class_='time')

    # Get the odds for the event
    odds = element.find_all('span', class_='odds')

    # Print the data
    print(event_name, event_time, odds)


In [ ]:
elements = soup.find_all('div')


In [ ]:
import requests
from bs4 import BeautifulSoup

# Specify the URL of the page you want to scrape
url = 'https://parimatch.com/en/'

# Make a request to the URL
response = requests.get(url)

# Create a BeautifulSoup object from the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the elements that contain the data you want to scrape
elements = soup.find_all('div', class_='event-card')
import requests
from bs4 import BeautifulSoup

# Specify the URL of the page you want to scrape
url = 'https://parimatch.com/en/'

# Make a request to the URL
response = requests.get(url)

# Create a BeautifulSoup object from the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the elements that contain the data you want to scrape
elements = soup.find_all('div', class_='event-card')

# Loop over the elements and extract the data
for element in elements:
    # Get the name of the event
    event_name = element.find('h3').text

    # Get the time of the event
    event_time = element.find('span', class_='time').text

    # Get the odds for the event
    odds = element.find_all('span', class_='odds')

    # Print the data
    print(event_name, event_time, odds)

    # Save the data to a file
    with open('parimatch_data.csv', 'w') as f:
        f.write('event_name,event_time,odds\n')
        for event_name, event_time, odds in zip(event_names, event_times, odds):
            f.write('{event_name},{event_time},{odds}\n'.format(event_name=event_name, event_time=event_time, odds=', '.join(odds)))
(elements)
# Loop over the elements and extract the data
for element in elements:
    # Get the name of the event
    event_name = element.find('h3').text

    # Get the time of the event
    event_time = element.find('span', class_='time').text

    # Get the odds for the event
    odds = element.find_all('span', class_='odds')

    # Print the data
    print(event_name, event_time, odds)

    # Save the data to a file
    with open('parimatch_data.csv', 'w') as f:
        f.write('event_name,event_time,odds\n')
        for event_name, event_time, odds in zip(event_names, event_times, odds):
            f.write('{event_name},{event_time},{odds}\n'.format(event_name=event_name, event_time=event_time, odds=', '.join(odds)))
